In [45]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# **Домашнее задание к уроку 3**

### Задание 1*

Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log. 

In [46]:
# Признаки
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
              800, 1500, 2000, 450, 1000], 
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype=np.float64)


In [47]:
# Целевая переменная
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [48]:
# Добавим в функцию малую константу limit для ограничения значений y_pred
def calc_logloss(y, y_pred):
    limit = 1e-15
    y_pred = pd.Series(y_pred)
    y_pred[y_pred < limit] = limit
    y_pred[y_pred > 1 - limit] = 1 - limit
    y_pred = np.array(y_pred)

    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

### Задание 2

Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [49]:
# Функция для сигмоиды
def sigmoid(x):
    return  1/(1+np.exp(-x))

In [50]:
# Функция для модели логистической регрессии
def eval_model(X, y, iterations, alpha=1e-4):
    
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    w_list = [W.copy()]
    errors = []
    
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        old_W = W.copy()
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        
        w_list.append(W.copy())
        errors.append(err)
        
        if i % (iterations / 10) == 0:
            print(i, W, err)
            
        if np.linalg.norm(W - old_W, ord=2) < 1e-3:
            z = np.dot(W, X)
            y_pred = sigmoid(z)
            err = calc_logloss(y, y_pred)
            errors.append(err)
            print(i, W, err)
            break
            
    w_list = np.array(w_list)
    
    return W, w_list, errors

In [51]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [52]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X_st[2, :])
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

In [53]:
# Предсказанные значения
y_pred = [0.1, 0.7, 0.85, 0, 0.9, 0.67, 0.15, 0.25, 0.6, 0.4]

In [54]:
alpha = 0.5
W, w_list, errors = eval_model(X_st, y, 30000, alpha)

3000 [-13.9452139   -1.673896    -3.20793787  11.64783166] 0.22733574187276795
6000 [-20.12251585  -2.20572082  -5.06399074  16.65680198] 0.18224925407755652
9000 [-24.75120926  -2.61226846  -6.42565532  20.41022448] 0.15709608251813184
12000 [-28.46628442  -2.94037977  -7.50619849  23.41754634] 0.1409623390833617
15000 [-31.58124983  -3.21545009  -8.40617794  25.93371632] 0.12965759007092087
18000 [-34.27538573  -3.45253656  -9.18157943  28.10488195] 0.1212246774064343
20730 [-36.45578216  -3.6433829   -9.80789023  29.85789089] 0.11516863770270366


In [55]:
alpha = 10
W, w_list, errors = eval_model(X_st, y, 3000, alpha)

300 [-57.59945273  -6.0838207  -15.38014682  47.81120021] 0.07904989896443981
600 [-62.79161593  -6.43888843 -16.97780213  51.72711074] 0.07263927640417525
900 [-65.14362793  -5.91609976 -18.16490873  52.35085478] 0.06945920539064297
1200 [-66.94708123  -6.03387704 -18.70899601  53.71366294] 0.06765212820981195
1500 [-68.68942557  -6.15080688 -19.23169057  55.03600988] 0.06596129910694601
1800 [-70.3801781   -6.26282276 -19.73998195  56.3163663 ] 0.06437132535501192
2100 [-72.02413057  -6.37049774 -20.23512179  57.55886797] 0.06286997154323497
2400 [-73.62524575  -6.47433549 -20.71812011  58.766988  ] 0.06144730689617424
2700 [-75.18685615  -6.57475619 -21.18982927  59.94363873] 0.06009516135437114
3000 [-76.71179501  -6.67211135 -21.65097857  61.09128407] 0.058806728427079215


In [56]:
alpha = 50
W, w_list, errors = eval_model(X_st, y, 1000, alpha)

100 [-178.13248818  -94.62862631   17.67730169  234.10762062] 6.922501137441179
200 [-298.95925808  -78.7586915   -61.26512012  252.24972248] 1.5932481680835433
300 [-305.64874175  -39.75753713  -68.34008646  274.92635567] 0.006055131891046036
400 [-306.31829132  -39.6828874   -69.57987794  274.25680607] 0.005558778386466649
500 [-306.80891789  -39.81576876  -70.66466894  273.76617951] 0.00522222815563359
600 [-307.22174796  -40.01725311  -71.66284383  273.35334943] 0.00494611083044943
700 [-307.58859478  -40.24670675  -72.59806807  272.98650261] 0.0047064916066530075
800 [-307.92384047  -40.48741589  -73.48247936  272.65125692] 0.004493276519686627
900 [-308.23533315  -40.73131957  -74.3236743   272.33976424] 0.004300863592157091
1000 [-308.52789676  -40.97418619  -75.12700074  272.04720063] 0.0041256137090789615


In [57]:
alpha = 100
W, w_list, errors = eval_model(X_st, y, 3000, alpha)

300 [-614.07504552  -85.0246012  -140.43658466  554.23587679] 0.00015412010709176769
600 [-614.24043247  -84.9083089  -140.6751444   554.07048983] 0.0001498982833832429
825 [-614.35202382  -84.84850849 -140.83847255  553.95889848] 0.00014743311145741953


**Вывод:** Для модели логистической регрессии подобрать оптимальные параметры крайне сложно. Градиентный спуск, использующийся в этой модели может очень долго стремиться к минимуму, но не достигнуть его. Скорее всего это вызвано тем, что функция логарифма не может принимать минимального значения.



### Задание 3

Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [58]:
# Функция для расчёта вероятности
def calc_pred_proba(W, X):
    return sigmoid(np.dot(W, X))

In [67]:
# Проверка
y_pred_pr = calc_pred_proba(W, X_st)
y_pred_pr

array([6.90283094e-04, 4.08442000e-29, 1.00000000e+00, 1.67968854e-16,
       1.00000000e+00, 7.01843982e-05, 1.00000000e+00, 1.40373763e-43,
       9.99286632e-01, 1.00000000e+00])

### Задание 4



Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).



In [81]:
# Функция для предсказанного класса
def calc_pred(W, X):
    y_pred_pr = calc_pred_proba(W, X)
    y_pred = np.array(list(map(int, y_pred_pr >= 0.5)))
    return y_pred

In [82]:
# Проверка
y_pred_4 = calc_pred(W1, X_st)
y_pred_4

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

### Задание 5

Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score

In [83]:
# Функция для вычисления accuracy
def accuracy(y, y_pred):
    result = np.array(list(map(int, y_pred == y)))
    return result.mean()

In [84]:
# Функция для расчёта матрицы ошибок
def confusion_matrix(y, y_pred):
    tp = np.array(list(map(int, (y_pred == 1) * (y == 1)))).sum()
    tn = np.array(list(map(int, (y_pred == 0) * (y == 0)))).sum()
    fp = np.array(list(map(int, (y_pred == 1) * (y == 0)))).sum()
    fn = np.array(list(map(int, (y_pred == 0) * (y == 1)))).sum()
    matrix = pd.DataFrame({'y=1': [tp, fn],
                          'y=0': [fp, tn]}, 
                          index=['y_pred=1', 'y_pred=0'])
    return matrix

In [75]:
# Функция для расчёта точности
def precision(y, y_pred):
    m = confusion_matrix(y, y_pred)
    return m.loc['y_pred=1', 'y=1'] / m.loc['y_pred=1'].sum()

In [76]:
# Функция для расчёта полноты
def recall(y, y_pred):
    m = confusion_matrix(y, y_pred)
    return m.loc['y_pred=1', 'y=1'] / m['y=1'].sum()

In [78]:
# Функция для расчёта F1 score
def f1_score(y, y_pred):
    result = 2 * precision(y, y_pred) * recall(y, y_pred) / (precision(y, y_pred) + recall(y, y_pred))
    return result

In [92]:
# accuracy
accuracy(y, y_pred_4)

1.0

In [93]:
# матрица ошибок
confusion_matrix(y, y_pred_4)

,y=1,y=0
y_pred=1,5,0
y_pred=0,0,5


In [94]:
# полнота
recall(y, y_pred_4)

1.0

In [95]:
# точность
precision(y, y_pred_4)

1.0

In [96]:
# F1 score
f1_score(y, y_pred_4)

1.0

### Задание 6

Могла ли модель переобучиться? Почему?

Модель может переобучиться по следующим причинам: у функции logloss нету минимума (т.к. в неё входит логарифм), при увеличении весов происходит уменьшение ошибки.